## Features variabelen uitbreiden 

In [1]:
# libraries importeren 
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# dataset inlezen 
df = df = pd.read_csv('C:\Amaryllis_school_documents\Jaar_20_21\DDB\ddb_prorail\DDB_Prorail\data\sap_storing_data_hu_project.csv', low_memory=False)

In [3]:
#Doel : en bepaald probleem op een bepaalde locatie op een bepaald tijdstip, een voorspelling te maken van de verwachte reparatietijd


#Belangrijke kolommen bepalen

# Geeft een goede definitie van de probleem 
#oorz_code : feature - nominaal 
#oorz_groep :feauture - nominaal 
#stm_prioriteit :feature - ordinaal 

# Deze kolom geven een duidelijke gebied waar de aannemer werkt.
#stm_contractgeb_mld of  stm_contractgeb_gst: C Gebied waar de aannemer werkt. - nominaal 
    # Definieer de speciefieke gebied waar de aannemer zit.
# geocode vallen in contract gebied , maar niet alle

# De tijdstip van een storing weergeven. 
# tijd van de melding 
data = df[['stm_oorz_groep', 'stm_oorz_code', 'stm_prioriteit','stm_contractgeb_mld','stm_contractgeb_gst','stm_sap_meldtijd', 'stm_aanntpl_dd','stm_aanntpl_tijd','stm_fh_ddt']].copy()

In [4]:
data.loc[:, 'reparatietijd'] = (pd.to_datetime(data['stm_fh_ddt']) -  pd.to_datetime(data.loc[:,'stm_aanntpl_dd']  +" "+ data['stm_aanntpl_tijd'])).dt.total_seconds().div(60)

In [5]:
data.head()

,stm_oorz_groep,stm_oorz_code,stm_prioriteit,stm_contractgeb_mld,stm_contractgeb_gst,stm_sap_meldtijd,stm_aanntpl_dd,stm_aanntpl_tijd,stm_fh_ddt,reparatietijd
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ONR-RIB,133.0,9.0,NaN,81.0,09:00:00,NaN,00:00:00,02/01/2006 09:00:00,NaN
2,ONR-DERD,143.0,9.0,NaN,37.0,12:35:00,NaN,00:00:00,02/01/2006 13:26:00,NaN
3,ONR-DERD,142.0,9.0,NaN,32.0,16:40:00,NaN,00:00:00,02/01/2006 17:20:00,NaN
4,ONR-DERD,142.0,9.0,NaN,35.0,22:30:00,NaN,00:00:00,02/01/2006 22:36:00,NaN


In [6]:
data = data.drop(['stm_aanntpl_dd','stm_aanntpl_tijd','stm_fh_ddt'], axis= 1)

We hebben te maken met een continu target variabel en categorical feature variabelen 

In [7]:
#Features variabelen  2:
data2 = data[['stm_oorz_groep', 'stm_oorz_code', 'stm_prioriteit', 'stm_contractgeb_gst', 'stm_sap_meldtijd', 'reparatietijd']].copy()
data2.head()

,stm_oorz_groep,stm_oorz_code,stm_prioriteit,stm_contractgeb_gst,stm_sap_meldtijd,reparatietijd
0,NaN,NaN,NaN,NaN,NaN,NaN
1,ONR-RIB,133.0,9.0,81.0,09:00:00,NaN
2,ONR-DERD,143.0,9.0,37.0,12:35:00,NaN
3,ONR-DERD,142.0,9.0,32.0,16:40:00,NaN
4,ONR-DERD,142.0,9.0,35.0,22:30:00,NaN


Voor de tijd van de melding moeten de tijden worden afgerond naar uren.

In [8]:
df2  = df[[ 'stm_contractgeb_gst', 'stm_sap_meldtijd' ]].copy()
df2.head()

,stm_contractgeb_gst,stm_sap_meldtijd
0,NaN,NaN
1,81.0,09:00:00
2,37.0,12:35:00
3,32.0,16:40:00
4,35.0,22:30:00


In [9]:
# De dimensie van de df2 bekijken
df2.shape

(898526, 2)

In [10]:
#datatypes bepalen
df2.dtypes

stm_contractgeb_gst    float64
stm_sap_meldtijd        object
dtype: object

In [11]:
#kijken hoeveel Nan waardes er zijn.
df2.isnull().sum()

stm_contractgeb_gst    223
stm_sap_meldtijd        38
dtype: int64

In [12]:
# bereken aantal rijen die wordt verloren bij Nan waardes, kan denk ik gewoon verwijderd worden, is maar een heel klein aantal.
print('Rijen verloren door Nan contractgebied : {}%'.format((df2['stm_contractgeb_gst'].isnull().sum()/898526)*100))
print('Rijen verloren door Nan meldtijd  : {}%'.format((df2['stm_sap_meldtijd'].isnull().sum()/898526)*100))
print('Totaal aantal rijen verloren : {} %'.format(df2.isnull().sum().sum()/898526*100))

Rijen verloren door Nan contractgebied : 0.02481842484246421%
Rijen verloren door Nan meldtijd  : 0.004229148627863857%
Totaal aantal rijen verloren : 0.02904757347032807 %


In [13]:
#de rijen daadwerkelijk verwijderen.

In [14]:
# Nog even kijken of er codes in het excel bestand staan die hier missen en andersom.
df2['stm_contractgeb_gst'][df2['stm_contractgeb_gst'].notnull()].sort_values().unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 50.,
       51., 52., 53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63.,
       64., 70., 71., 81., 82., 83., 99.])

Er was in de excel bestand ekeken en de volgende codes kwamen niet voor:
 50., 53., 54., 55., 56., 57., 62., 64., 82., 83.

In [15]:
len(df2['stm_contractgeb_gst'].unique())

60

In [16]:
# 50., 53., 54., 55., 56., 57., 62., 64., 82., 83.
a = len(df2[df2['stm_contractgeb_gst'] == 50])
a

348

In [17]:
b = len(df2[df2['stm_contractgeb_gst'] == 53])
b

15073

In [18]:
c = len(df2[df2['stm_contractgeb_gst'] == 54])
c

5926

In [19]:
d = len(df2[df2['stm_contractgeb_gst'] == 55])
d

5253

In [20]:
e = len(df2[df2['stm_contractgeb_gst'] == 56])
e

2898

In [21]:
f =len(df2[df2['stm_contractgeb_gst'] == 57])
f

289

In [22]:
g = len(df2[df2['stm_contractgeb_gst'] == 62])
g

9377

In [23]:
h = len(df2[df2['stm_contractgeb_gst'] == 64])
h

4051

In [24]:
i = len(df2[df2['stm_contractgeb_gst'] == 82])
i

111

In [25]:
j = len(df2[df2['stm_contractgeb_gst'] == 83])
j

694

In [26]:
totaal = a + b + c + d + e + f + g + h + i + j
totaal

44020

In [27]:
# aantal waardes dat verloren zou gaan als we het verwijderen. (in procenten)
44020/898526*100

4.899134805225447

## Data Preparation

In [29]:
# Alle Nan waardes verwijderen 
data2 = data2.dropna()

In [30]:
#Controle
data2.isnull().sum()

stm_oorz_groep         0
stm_oorz_code          0
stm_prioriteit         0
stm_contractgeb_gst    0
stm_sap_meldtijd       0
reparatietijd          0
dtype: int64

In [31]:
#niet behorende contract gebied codes verwijderen:[ 50., 53., 54., 55., 56., 57., 62., 64., 82., 83.]
data2 = data2[(data2['stm_contractgeb_gst'].isin([50, 53, 54, 55, 56, 57, 62, 64, 82, 83]))== False]

In [32]:
#we verwijdert alles behalve de laatse , omdat dit als laats toegevoegd(meeste recente storing).????
print(len(data2[data2.duplicated(keep='last') == False]))
data2 = data2[data2.duplicated(keep='last') == False]

424654


In [33]:
# oorzaak groep, ooorzaak code en contract gebied codes encoden
data2 = pd.get_dummies(data2, columns = ['stm_oorz_groep'])
data2 = pd.get_dummies(data2, columns = ['stm_oorz_code'])
data2 = pd.get_dummies(data2, columns = ['stm_contractgeb_gst'])

In [34]:
#Bij et omzetten naat datetime kregennn we een error dtatt '::' niet kan omgezet worden. We gaan nu kijken naar alle ijen die '::'
#als meldtijd heeft.
data2[data2['stm_sap_meldtijd']=="::"].head()

,stm_prioriteit,stm_sap_meldtijd,reparatietijd,stm_oorz_groep_ONR-DERD,stm_oorz_groep_ONR-RIB,stm_oorz_groep_TECHONV,stm_oorz_groep_WEER,stm_oorz_code_33.0,stm_oorz_code_51.0,stm_oorz_code_130.0,...,stm_contractgeb_gst_52.0,stm_contractgeb_gst_58.0,stm_contractgeb_gst_59.0,stm_contractgeb_gst_60.0,stm_contractgeb_gst_61.0,stm_contractgeb_gst_63.0,stm_contractgeb_gst_70.0,stm_contractgeb_gst_71.0,stm_contractgeb_gst_81.0,stm_contractgeb_gst_99.0
145284,4.0,::,13.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
228778,5.0,::,29.0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
234564,1.0,::,85.0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
238027,2.0,::,5.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
239817,2.0,::,75.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
# aantal rijen die '::' als melldtijd heeft
len(data2[data2['stm_sap_meldtijd']=="::"])

49

In [36]:
#Deze 49 rijen kunne we uit onze data uithalen
#als meldtijd heeft.
data2 = data2[data2['stm_sap_meldtijd']!="::"]

In [37]:
# Tijd van de melding moet afgerond worden ,van  een string en in de format hh:mm:ss, 
#naar datetime/int en in de format uren. 

data2['stm_sap_meldtijd_uren'] = round(pd.to_timedelta(data2['stm_sap_meldtijd']).dt.total_seconds().div(3600)).astype(int)
#data2['stm_sap_meldtijd_uren'] = data2['stm_sap_meldtijd_uren'].dt.total_seconds().div(3600)

In [38]:
data2.head()

,stm_prioriteit,stm_sap_meldtijd,reparatietijd,stm_oorz_groep_ONR-DERD,stm_oorz_groep_ONR-RIB,stm_oorz_groep_TECHONV,stm_oorz_groep_WEER,stm_oorz_code_33.0,stm_oorz_code_51.0,stm_oorz_code_130.0,...,stm_contractgeb_gst_58.0,stm_contractgeb_gst_59.0,stm_contractgeb_gst_60.0,stm_contractgeb_gst_61.0,stm_contractgeb_gst_63.0,stm_contractgeb_gst_70.0,stm_contractgeb_gst_71.0,stm_contractgeb_gst_81.0,stm_contractgeb_gst_99.0,stm_sap_meldtijd_uren
19177,9.0,07:23:00,0.0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
19220,9.0,07:00:00,0.0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
20735,9.0,19:38:00,0.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20
21228,9.0,07:12:00,0.0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
22779,9.0,14:00:00,1.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,14


In [39]:
# Data selectie waar we alle rijen pakken di een reparatietijd heeft 
data2 = data2[(data2['reparatietijd'] >= 15) & (data2['reparatietijd'] <=  480)]

In [44]:
data2.describe()

,stm_prioriteit,reparatietijd,stm_oorz_groep_ONR-DERD,stm_oorz_groep_ONR-RIB,stm_oorz_groep_TECHONV,stm_oorz_groep_WEER,stm_oorz_code_33.0,stm_oorz_code_51.0,stm_oorz_code_130.0,stm_oorz_code_131.0,...,stm_contractgeb_gst_58.0,stm_contractgeb_gst_59.0,stm_contractgeb_gst_60.0,stm_contractgeb_gst_61.0,stm_contractgeb_gst_63.0,stm_contractgeb_gst_70.0,stm_contractgeb_gst_71.0,stm_contractgeb_gst_81.0,stm_contractgeb_gst_99.0,stm_sap_meldtijd_uren
count,283710.000000,283710.000000,283710.000000,283710.000000,283710.000000,283710.000000,283710.000000,283710.0,283710.000000,283710.000000,...,283710.000000,283710.000000,283710.000000,283710.000000,283710.000000,283710.000000,283710.000000,283710.000000,283710.000000,283710.000000
mean,3.744820,77.957637,0.191724,0.105640,0.654080,0.048557,0.000004,0.0,0.000148,0.000201,...,0.007927,0.006521,0.003863,0.008770,0.006485,0.000243,0.026055,0.017694,0.000712,12.445684
std,1.383249,75.357940,0.393658,0.307376,0.475668,0.214940,0.001877,0.0,0.012166,0.014173,...,0.088681,0.080488,0.062034,0.093234,0.080271,0.015593,0.159299,0.131837,0.026674,5.612488
min,1.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,30.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000
50%,4.000000,52.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000
75%,5.000000,95.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,16.000000
max,9.000000,480.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,24.000000


In [49]:
# stm_sap_meldtijd verwijderen
data2 = data2.drop('stm_sap_meldtijd', axis  = 1)

In [50]:
#Sla data2 op in een pkl bestand  
data2.to_pickle("data2.pkl")